# DATA20001 Deep Learning - Group Project
## Image project


- Testing first implementation of [resnext50_32x4d](https://pytorch.org/docs/stable/torchvision/models.html#id27) pretrained transfer learning model
- Data preprocessing includes data augmentation, shuffle and split train=17k, val=1.5k, test=1.5k
- Data augmentation IS applied
- No data balancing applied



In [1]:
# automatically reload dependencies and repository content so that kernel need not be restarted
%load_ext autoreload
%autoreload 2

## Import dependencies and select correct device

In [2]:
# import dependencies
import train_model
import eval_model
import utils
from image_dataset import ImageDataset
from data_augmentation import DataAugmentation
from data_balancer import DataBalancer

import time
import torch
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [3]:
device = None
if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda:0")
else:
    print("Using CPU")
    device = torch.device("cpu")

Using CPU


## Define datasets

- shuffle
- train: 17k
- valid: 1.5k
- test: 1.5k

In [4]:
mapping_file = "file_to_labels_table.csv"
df = pd.read_csv(mapping_file)
train_idx, val_idx = utils.get_train_val_indexes(df, 0.85, shuffle=True)
val_test_split_idx = int(val_idx.shape[0]*.5)
test_idx = val_idx[:val_test_split_idx]
val_idx = val_idx[val_test_split_idx:]

dataAugmentation = DataAugmentation()
trainIds = dataBalancer.balanceData(range(1,19001), 80000-19000, range(1, 19001))
trainDataset = ImageDataset(train_idx, dataAugmentation=dataAugmentation)
valDataset = ImageDataset(val_idx, dataAugmentation=dataAugmentation)
testDataset = ImageDataset(test_idx, dataAugmentation=dataAugmentation)

print("Saving test dataset temporarily so test can be performed after restarting kernel.")
utils.save_pickle(testDataset, "tmp_test_data_exp_transfer_learning_2.pkl")
print("# Train:", len(trainDataset))
print("# Valid:", len(valDataset))
print("# Test:", len(testDataset))

Saving test dataset temporarily so test can be performed after restarting kernel.
# Train: 17000
# Valid: 1500
# Test: 1500


## Define the model architecture

The initial transfer learning implementation

In [4]:
preTrainedModel = models.resnext50_32x4d(pretrained=True)

for param in preTrainedModel.parameters():
    param.requires_grad = False

num_ftrs = preTrainedModel.fc.in_features
num_classes = 14

preTrainedModel.fc = torch.nn.Sequential(
    torch.nn.Linear(num_ftrs, num_classes),
    torch.nn.Sigmoid()
)

model = preTrainedModel.to(device)

## Define test case

Hyperparam search could be defined here. No hyperparam searches in this notebook as purpose of this test is to test the effect of data preprocessing technique on initial transfer learning model architecture and see how longer run on epochs affect loss.

**Test objective**:

- Testing first implementation of [resnext50_32x4d](https://pytorch.org/docs/stable/torchvision/models.html#id27) pretrained transfer learning model
- Data preprocessing includes data augmentation, shuffle and split train=17k, val=1.5k, test=1.5k
- Data augmentation IS applied
- No data balancing applied

**Training results**:

- Training job ran over 19.5 taking ~25 hours
- The job ran on the background and jupyter client was not connected the whole time

In [ ]:
n_epochs = 25

model, logs = train_model.train_model(
    model,
    trainDataset,
    valDataset,
    device,
    numberOfEpochs=n_epochs,
    return_logs=True
)

## Save model and logs

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
pd.DataFrame(logs).to_csv("exp_transfer_learning_2_logs.csv", index=False)
torch.save(model.state_dict(), 'exp_transfer_learning_2_model.pkl')

## Test model

- Load model back from state_dict and perform test on testing set
- Load test-set
- Load training time logs of performance

In [5]:
model.load_state_dict(torch.load("exp_transfer_learning_2_model.pkl"))
testDataset = utils.load_pickle("tmp_test_data_exp_transfer_learning_2.pkl")
train_logs = pd.read_csv("exp_transfer_learning_2_logs.csv")
y_hats, y_trues = eval_model.test_model(model, testDataset, device)

### Validation set performance during training

In [6]:
train_logs

,epoch,accuracy,loss
0,0,"tensor(0.9496, dtype=torch.float64)",0.141267
1,1,"tensor(0.9513, dtype=torch.float64)",0.132198
2,2,"tensor(0.9540, dtype=torch.float64)",0.131344
3,3,"tensor(0.9533, dtype=torch.float64)",0.131660
4,4,"tensor(0.9555, dtype=torch.float64)",0.126070
5,5,"tensor(0.9541, dtype=torch.float64)",0.132351
6,6,"tensor(0.9541, dtype=torch.float64)",0.134154
7,7,"tensor(0.9540, dtype=torch.float64)",0.132241
8,8,"tensor(0.9511, dtype=torch.float64)",0.149421
9,9,"tensor(0.9472, dtype=torch.float64)",0.158580


### Testing set performance

In [7]:
print("Testing set results.")
print(f" Accuracy\t{round(eval_model.get_metric(y_trues, y_hats, 'accuracy'), 3)}")
print(f" Precision\t{round(eval_model.get_metric(y_trues, y_hats, 'precision'), 3)}")
print(f" Recall \t{round(eval_model.get_metric(y_trues, y_hats, 'recall'), 3)}")
print(f" F1-score\t{round(eval_model.get_metric(y_trues, y_hats, 'f1'), 3)}")

Testing set results.
 Accuracy	0.96
 Precision	0.739
 Recall 	0.683
 F1-score	0.71


### Extra analysis

How many positives and negatives has been predicted.

Tuples like: **(label, count)**

In [8]:
list(zip(*np.unique(y_hats, return_counts=True)))

[(0, 19592), (1, 1408)]

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')